In [168]:
using LinearAlgebra
#APOIOS

#
# Aplica condições de contorno essenciais 
# homogêneas
#
function Aplica_CC_homo!(apoios,K,F,G)

    # Aplica as condições de contorno essenciais homogêneas
    for i=1:size(apoios,1) 

        # No e gl local do apoio
        no  = Int(apoios[i,1])
        gll = Int(apoios[i,2])

        # Testa se o usuário tem ideia do que
        # ele está fazendo
        if apoios[i,3]!=0 
            error("Sua anta")
        end

        # Gl global
        gl = 2*(no-1) + gll

        # Zera linha e coluna da rigidez
        K[gl,:] .= 0.0
        K[:,gl] .= 0.0

        # Zera o vetor de forças nesse gl
        F[gl] = 0.0

        #Zera o vetor G nesse gl
        G[gl,:] .= 0.0

        # Coloca 1.0 na diagonal
        K[gl,gl] = 1.0

    end #gls

end

Aplica_CC_homo! (generic function with 1 method)

In [211]:
#GLOBAL 

#
# conectividades é uma matriz com ne linhas (número de elementos)
# e 2 colunas (nó inicial e nó final)
#
function Global(dim,ne,nnos,conectividades,coord, VE,Vnuxy)

    # Precisamos definir a matriz global
    K = zeros(dim,dim)
    # Aloca vetores para as coordenadas de cada elemento
    X = zeros(4)
    Y = zeros(4)
    nos = zeros(4)
    gls = zeros(Int,8)

    # Loop nos elementos da malha
    for ele=1:ne       
    
        # Recupera as informações do elemento
        Ee = VE[ele]
        ve = Vnuxy[ele]
        
        # Monta a matriz constitutiva do elemento
        C = Calcula_C(Ee,ve,"INC") #alterei o C para D com a hipotese de incompressibilidade.
        
        # Recupera as coordenadas do elemento
        for i=1:4
            # Descobre o nó
            no = conectividades[ele,i]
            
            # Guarda os nós do elemento
            nos[i] = no
        
            # Descobre as coordenadas
            X[i],Y[i] = coord[no,:]
        end
        
        # Monta a matriz de rigidez do elemento
        # no sistema local, com o elementos escolhido
        
        Kg = K_up(C,X,Y)

        # Agora precisamos posicionar Kg na matriz global do problema
        
        # Vetor com os gls GLOBAIS do elemento
        contador = 1
        # Varre os nós
        for i=1:4
            # Varre os gls do nó
            for j=1:2
                gls[contador] = Int(2*(nos[i]-1)+j)
                contador += 1
            end
        end
        # Soma Kg nas posições gls em K
        K[gls,gls] .= K[gls,gls] .+ Kg
        
    end #ele

    
    return K
end

#
# Monta o vetor de força global
#
function Forca_global(nnos,forcas)

    # Montar o vetor de forças global
    F = zeros(2*nnos)

    # Para cada informação em forças 
    # posiciona a força no vetor de forças
    # globais do problema
    for i=1:size(forcas,1)
       
        # Recupera nó e gl local 
        no  = Int(forcas[i,1])
        gll = Int(forcas[i,2])

        # Recupera valor
        valor = forcas[i,3]

        # Adiciona ao valor da força
        F[2*(no-1)+gll] += valor

    end #i

    return F
end

function Gglobal(dim,ne,nnos,conectividades,coord, VE,Vnuxy)
    # Aloca vetores para as coordenadas de cada elemento
    X = zeros(4)
    Y = zeros(4)
    nos = zeros(4)
    gls = zeros(Int,8) 
    G = zeros(2*nnos,ne) #12,2

    contador = 1
    # Loop nos elementos da malha
    for ele=1:ne

        # Recupera as informações do elemento
        Ee = VE[ele]
        ve = Vnuxy[ele]
    
        # Recupera as coordenadas do elemento
        for i=1:4
            # Descobre o nó
            no = conectividades[ele,i]
            
            # Guarda os nós do elemento
            nos[i] = no
    
            # Descobre as coordenadas
            X[i],Y[i] = coord[no,:]
        end
    
        # Monta a matriz G
        Ge = G_up(X,Y)
        
        # Vetor com os gls GLOBAIS do elemento
        contador = 1
        # Varre os nós
        for i=1:4
            # Varre os gls do nó
            for j=1:2
                gls[contador] = Int(2*(nos[i]-1)+j)
                contador += 1
            end
        end
        
        # Soma Kg nas posições gls em K
        G[gls,ele] .= G[gls,ele] .+ Ge

        
    end #ele
        
    return G
end

function Mglobal(dim,ne,nnos,conectividades,coord,VE,Vnuxy)
    # Aloca vetores para as coordenadas de cada elemento
    X = zeros(4)
    Y = zeros(4)
    nos = zeros(4)
    gls = zeros(Int,dim)
    M = zeros(ne,ne)

    # Loop nos elementos da malha
    for ele=1:ne
        
        # Recupera as informações do elemento
        Ee = VE[ele]
        ve = Vnuxy[ele]
    
        # Recupera as coordenadas do elemento
        for i=1:4
            # Descobre o nó
            no = conectividades[ele,i]
            
            # Guarda os nós do elemento
            nos[i] = no
    
            # Descobre as coordenadas
            X[i],Y[i] = coord[no,:]
        end
    
        # Monta a matriz G
        a = M_up(X,Y,Ee,ve)
        M[ele, ele] = M_up(X,Y,Ee,ve)
        
    end
    
    return M
end


Mglobal (generic function with 1 method)

In [212]:
#MATERIAL 

# Monta a matriz constitutiva dado E,v e a hipótese
function Calcula_C(E,v,hipotese="INC")

    # Testa se hipotese é válida
    hipotese in ["EPT","EPD", "INC"] || throw("Hipotese errada")
    
    G = E/(2*(1+v))

    # adicionando trecho incompressibilidade -
       
    if hipotese=="EPT"
        a = E/(1-v^2)
        C = [a v*a 0.0 ;
            v*a a  0.0 ;
              0  0  G]

    elseif hipotese=="EPD"
        d = (v+1)*(2*v-1)  
        a = E*(v-1)/d
        b = -(E*v)/d
        C = [a b 0 ;
             b a 0 ;
             0 0 G]
        
    elseif hipotese=="INC"
        μ = E/(2*(1+v))
        
        C = [2*μ 0    0 ;
              0   2*μ  0 ;
              0   0    μ ]
        
    else
 
        C = [a b b 0 0 0 ;
             b a b 0 0 0 ;
             b b a 0 0 0 ;
             0 0 0 G 0 0 ;
             0 0 0 0 G 0 ;
             0 0 0 0 0 G]

    end
    
    return C

end


Calcula_C (generic function with 2 methods)

In [213]:
#MATERIAL

# N1 = 0.25(1-r)(1-s)
# N2 = 0.25(1+r)(1-s)
# N3 = 0.25(1+r)(1+s)
# N4 = 0.25(1-r)(1+s)

#
# Calcula a matriz B e o determinante da matriz
# Jacobiana no ponto (r,s) 
#

#B para o procedimento no elemento u-p é o mesmo que dos elementos normaais.
function B_up(r,s,X,Y)

   # Vetores com as derivadas das funções de interpolação
   # em relação a r e a s
   dNr = [ -0.25*(1-s) ; 0.25*(1-s) ; 0.25*(1+s)  ; -0.25*(1+s)   ]
   dNs = [ -0.25*(1-r) ;-0.25*(1+r) ; 0.25*(1+r)  ;  0.25*(1-r)   ]

   # Aloca e calcula a matriz J
   J = zeros(2,2)

   # Loop do somatório para cada posição de J
   for i=1:4
       J[1,1] += dNr[i]*X[i]
       J[1,2] += dNr[i]*Y[i]
       J[2,1] += dNs[i]*X[i]
       J[2,2] += dNs[i]*Y[i]
   end #i

   # Calcula o determinante
   dJ = det(J)

   # Calcula a inversa da J
   invJ = inv(J)

   # Aloca a matriz B
   B = zeros(3,8)

   # Loop pelos blocos da B, com as correções de derivadas
   c = 1
   for i=1:4
     
       # Correção das derivadas para este bloco
       dNxy = invJ*[dNr[i];dNs[i]]

       # Posiciona na B
       B[1,c]   = dNxy[1]
       B[2,c+1] = dNxy[2]
       B[3,c]   = dNxy[2]
       B[3,c+1] = dNxy[1]

       # Atualiza a apontador de bloco
       c += 2

   end #i

   return dJ, B
end

function Bb_up(r,s,X,Y) 

   # Vetores com as derivadas das funções de interpolação
   # em relação a r e a s
   dNr = [ -0.25*(1-s) ; 0.25*(1-s) ; 0.25*(1+s)  ; -0.25*(1+s)   ]
   dNs = [ -0.25*(1-r) ;-0.25*(1+r) ; 0.25*(1+r)  ;  0.25*(1-r)   ]

   # Aloca e calcula a matriz J
   J = zeros(2,2)

   # Loop do somatório para cada posição de J
   for i=1:4
       J[1,1] += dNr[i]*X[i]
       J[1,2] += dNr[i]*Y[i]
       J[2,1] += dNs[i]*X[i]
       J[2,2] += dNs[i]*Y[i]
   end #i

   # Calcula o determinante
   dJ = det(J)

   # Calcula a inversa da J
   invJ = inv(J)

   # Aloca a matriz Bbolinha
   B = zeros(8,1)

   # Loop pelos blocos da B, com as correções de derivadas
   c = 1

    #essa e a matriz Bbolinha que aplica o operador gradiente (∇)
   for i=1:4
        # Correção das derivadas para este bloco
        dNxy = invJ*[dNr[i];dNs[i]]
        
        # Posiciona na B
        B[c,1] = dNxy[1]
        B[c+1,1] = dNxy[2]
        
        # Atualiza a apontador de bloco
        c += 2

   end #i

   return dJ, B
end

function Mb_up(r,s,X,Y) 

   # Vetores com as derivadas das funções de interpolação
   # em relação a r e a s
   dNr = [ -0.25*(1-s) ; 0.25*(1-s) ; 0.25*(1+s)  ; -0.25*(1+s)   ]
   dNs = [ -0.25*(1-r) ;-0.25*(1+r) ; 0.25*(1+r)  ;  0.25*(1-r)   ]

   # Aloca e calcula a matriz J
   J = zeros(2,2)

   # Loop do somatório para cada posição de J
   for i=1:4
       J[1,1] += dNr[i]*X[i]
       J[1,2] += dNr[i]*Y[i]
       J[2,1] += dNs[i]*X[i]
       J[2,2] += dNs[i]*Y[i]
   end #i

   # Calcula o determinante
   dJ = det(J)

   # Calcula a inversa da J
   invJ = inv(J)

   # Loop pelos blocos da B, com as correções de derivadas

   return dJ
end


#
# Monta a matriz de rigidez de um elemento
# bilinear isoparamétrico de 4 nós para formulação u-p. 
#
function K_up(C::AbstractMatrix,X::Vector,Y::Vector)

    # Aloca a matriz do elemento. Quantos nós eu tenho por elemento?
    K = zeros(8,8)

    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]

    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
           s  = pg[j]  
           ws = W[j]

           # calcula o dJ e o B
           dJ, B = B_up(r,s,X,Y)

           # Acumula o produto nesses ptos
           K = K + B'*C*B*dJ

        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return K

end

#operador diferencial divergente

#
# Monta a matriz G

function G_up(X::Vector,Y::Vector)

    # Aloca a matriz do elemento
    G = zeros(8,1)

    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]
    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
            s  = pg[j]  
            ws = W[j]
            # calcula o dJ e o B
            dJ, B = Bb_up(r,s,X,Y)
            # Acumula o produto nesses ptos
            G = G + (B*dJ*1)
        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return G
end


function M_up(X::Vector,Y::Vector,E,ν) 

    # Aloca a matriz do elemento
    M = 0
    λ = (ν*E) / ((1+ν)*(1-2*ν))
    
    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]
    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
            s  = pg[j]  
            ws = W[j]
            # calcula o dJ e o B
            dJ = Mb_up(r,s,X,Y)
            
            # Acumula o produto apenas na diagonal principal
            M = M + ((1 / λ) * dJ)
        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return M

end


M_up (generic function with 1 method)

In [287]:
#função que roda todas as rotinas, estou dando de entrada a função que define o meu problema e o poisson, pra ficar facil de testar.
function main_inc(func::Function, v0)
    
    nnos,ne,dim,coord,conectividades,VE,Vnuxy,apoios,forcas,λ = func(v0)
    
    K = Global(dim,ne,nnos,conectividades,coord, VE, Vnuxy)
    M = Mglobal(dim,ne,nnos,conectividades,coord, VE, Vnuxy)
    F = Forca_global(nnos,forcas)
    G = Gglobal(dim,ne,nnos,conectividades,coord, VE, Vnuxy)

    # Modifica o sistema pela aplicação das condições de contorno
    # homogêneas
    Aplica_CC_homo!(apoios,K,F,G)
    #display(M)
    #Matriz H zerada:
    H = zeros(1,1)
    
    if Vnuxy[1]==0.5
        println("Caso incompressível, procedimento 2 Hughes: ")
        #M = zeros(2,2)

        #G'INV(K)G)p = G'INV(K)F-H
        #Estou considerando H como 0.
        leq = G'*(inv(K)*G)
        req = G'*(inv(K)*F) 

        p = leq\req
        #Kd + Gp = F
        #Kd = F - Gp 

        req2 = F - G*p
        d = K\req2

    else
        #(K-G*inv(M)*G')*d = F-G*inv(M)*H  # 4.3.25
        #leq = (K-G*inv(M)*G')
        #req = F-GM*H = F
        
        println("Caso compressível, procedimento 1 Hughes: ")
        
        leq = K+(G*(inv(M)*G'))
        #req = F - (G*inv(M)*H) como H = 0 req = F
        
        req = F #- (G*inv(M)*H)
        d = leq\req
        p = inv(M)*G'*d
    end


    return d,p,M,G,K
end

main_inc (generic function with 1 method)

In [290]:
using NBInclude
@nbinclude("casos.ipynb")

caso_g2 (generic function with 1 method)

In [291]:
d,p,m,g,K = main_inc(caso_g2,0.5)
imprimir_d(d)
imprimir_p(p)

Caso incompressível, procedimento 2 Hughes: 

Nó	Deslocamento em x	Deslocamento em y
1	   0.00000	             0.00000
2	  11.05549	            -2.93899
3	  39.13232	            -5.25896
4	  77.33793	            -5.83629
5	   0.00000	             0.00000
6	  11.06314	             2.96378
7	  39.11050	             5.23778
8	  77.39188	             5.92570
9	   0.00000	             0.00000
10	  10.61661	             0.01312
11	  39.21957	            -0.01109
12	  77.17185	             0.04074

Os dados de pressão para cada elemento: 

Elemento	Pressao
1	     -1890.383
2	        31.132
3	      -311.045
4	      1877.623
5	        -7.530
6	       257.481


In [292]:
imprimir_d(d)
imprimir_p(p)


Nó	Deslocamento em x	Deslocamento em y
1	   0.00000	             0.00000
2	  11.05549	            -2.93899
3	  39.13232	            -5.25896
4	  77.33793	            -5.83629
5	   0.00000	             0.00000
6	  11.06314	             2.96378
7	  39.11050	             5.23778
8	  77.39188	             5.92570
9	   0.00000	             0.00000
10	  10.61661	             0.01312
11	  39.21957	            -0.01109
12	  77.17185	             0.04074

Os dados de pressão para cada elemento: 

Elemento	Pressao
1	     -1890.383
2	        31.132
3	      -311.045
4	      1877.623
5	        -7.530
6	       257.481


In [239]:
using Printf

function imprimir_d(deslocamentos)
    n_nos = length(deslocamentos) ÷ 2
    # Cabeçalho da tabela
    println("\nNó\tDeslocamento em x\tDeslocamento em y")
    for i in 1:n_nos
        desloc_x = deslocamentos[2*i-1]
        desloc_y = deslocamentos[2*i]
        @printf("%d\t%10.5f\t%20.5f\n", i, desloc_x, desloc_y)
    end
end

function imprimir_p(pressao)
    n_nos = length(pressao)
    # Cabeçalho da tabela
    println("\nOs dados de pressão para cada elemento: \n")
    println("Elemento\tPressao")
    for i in 1:n_nos
        p = pressao[i]
        @printf("%d\t%14.3f\n", i, pressao[i])
    end
end


imprimir_p (generic function with 1 method)